## Import the libraries

In [3]:
%pip install datasets
%pip install accelerate
%pip install sentencepiece
%pip install --upgrade huggingface_hub
%pip -q install git+https://github.com/huggingface/transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement huggingface-hub<1.0,>=0.19.3 (from transformers) (from versions: 0.0.1, 0.0.2, 0.0.3rc1, 0.0.3rc2, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.1.0, 0.1.1, 0.1.2, 0.2.0, 0.2.1, 0.4.0)
ERROR: No matching distribution found for huggingface-hub<1.0,>=0.19.3
Note: you may need to restart the kernel to use updated packages.


In [5]:
from huggingface_hub import notebook_login
notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywdidgets` module: `pip install ipywidgets`.

## Load CSV File

In [3]:
from datasets import load_dataset

data_path = '/content/drive/MyDrive/MALIN/exos_to_csv.csv'
dataset = load_dataset('csv', data_files=data_path)

In [4]:
# Split the dataset into 3 sets for train, test and validation
train_testsplit = dataset['train'].train_test_split(test_size=0.2)
test_validsplit = train_testsplit['test'].train_test_split(test_size=0.5)

In [5]:
train_ds = train_testsplit['train']
valid_ds = test_validsplit['train']
test_ds = test_validsplit['test']

In [6]:
train_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 1022
})

In [7]:
valid_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 128
})

In [8]:
test_ds

Dataset({
    features: ['filename', 'consigne', 'enonce'],
    num_rows: 128
})

## Genertion Part

In [19]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [20]:
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Prepare the [MASK] token

In [28]:
import numpy as np

In [48]:
def mask_replace(text):
    mask_proba=0.15
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    tokenized_input = tokenizer.tokenize(text["consigne"])
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_input)
    nums_token = int(len(input_ids) * mask_proba)
    mask_indices = np.random.choice(len(input_ids), nums_token, replace=False)

    masked_input_ids = input_ids.copy()

    for idx in mask_indices:
    # Do not mask special token like CLS and SEP
      if tokenizer.convert_ids_to_tokens([input_ids[idx]])[0] in [tokenizer.cls_token, tokenizer.sep_token]:
        continue
      masked_input_ids[idx] = tokenizer.convert_tokens_to_ids([tokenizer.mask_token])[0]

    # Convert ids back to tokens
    masked_tokens = tokenizer.convert_ids_to_tokens(masked_input_ids)
    masked_text = tokenizer.convert_tokens_to_string(masked_tokens)

    return {"consigne_masked":masked_text}

In [47]:
train_ds = train_ds.map(lambda examples: mask_replace())
train_ds

Map:   0%|          | 0/1022 [00:00<?, ? examples/s]

TypeError: string indices must be integers

In [16]:
# Prompt method
def generate_consigne(corpus):

  prompt = "Generate the text according to the source given."
  # encode the input text
  encoded_input = tokenizer(corpus["consigne"], return_tensors="pt", truncation=True, max_length=512, padding=True).to(device)
  # generate
  generated_tokens = model.generate(**encoded_input)
  return {"generated_consigne": generated_tokens}